__качаем объявления о продаже квартир с циан.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import sys
import re
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import random

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [3]:
raw_data_path = 'data/raw/flat/cian' # каталог с файлами объявлений

In [4]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'
# base_url, req_param = 'https://sevastopol.cian.ru/kupit-kvartiru/', ''
place = 'sevastopol'
base_url, req_param ='https://www.cian.ru/cat.php', 'deal_type=sale&offer_type=flat&region=184723'

In [5]:
%%time

# from lib.downloader import DownloaderSimple
# from lib.parser import AdsListParser
from lib.downloader import DownloaderSeleniumFirefox
from lib.cian import CianParser


df,html = CianParser(
        base_url = base_url,
        driver=DownloaderSeleniumFirefox(profile_path),
        # driver=DownloaderSimple(),
    ).load(req_param=req_param, keep_html=True) # , npage_start=53) # , page_limit=3 ) #

print(len(df))

[INFO    ] 2023-01-10 11:21:26 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2023-01-10 11:21:26 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2023-01-10 11:21:29 | AdsListParser: downloader init
[INFO    ] 2023-01-10 11:21:29 | CianParser: downloader init
[INFO    ] 2023-01-10 11:21:29 | AdsListParser: start read and parse pages...
[INFO    ] 2023-01-10 11:21:39 | AdsListParser: read page 1
[INFO    ] 2023-01-10 11:21:50 | AdsListParser: read page 2
[INFO    ] 2023-01-10 11:22:03 | AdsListParser: read page 3
[INFO    ] 2023-01-10 11:22:18 | AdsListParser: read page 4
[INFO    ] 2023-01-10 11:22:30 | AdsListParser: read page 5
[INFO    ] 2023-01-10 11:22:40 | AdsListParser: read page 6
[INFO    ] 2023-01-10 11:22:47 | AdsListParser: read page 7
[INFO    ] 2023-01-10 11:22:55 | AdsListParser: read page 8
[INFO    ] 2023-01-10 11:23:06 | AdsListParser: read page 9
[INFO    ] 2023-01-10 11:23:18 | AdsListParser: read page 10
[INFO    ] 2023-01-10 11:23:31 | AdsLis

In [6]:
df.sample(3)

,OfferTitle,OfferSubtitle,Deadline,MainPrice,PriceInfo,GeoLabel,TimeLabel,LinkArea,Description,page,ts
437,"3-комн. кв., 51 м², 2/2 этаж",,,5 000 000 ₽,98 039 ₽/м²,"[Севастополь, р-н Балаклавский, с. Орлиное, улица Ивана Пахомова, 2]","[2 недели назад, 21 дек, 17:40]",https://sevastopol.cian.ru/sale/flat/271437043/,"ВНИМАНИЕ! ВНИМАНИЕ! Срочная продажа 3-х комнатной квартиры!\n В одном из самых чудесных мест Крыма в Байдарской долине в селе Орлиное (Крымская Швейцария) продается отличная квартира! Тишина и свежий горный воздух будут спутниками по жизни в заповедном месте. В селе находятся школа, садик, больница, МФЦ, банк РНКБ, администрация, магазины и продуктовые и хозяйственные, аптека, почта. До Севастополя, как и до Ялты 30 мин на транспорте. Городские автобусы ходят каждые полчаса. До Байдарских ворот (знаменитый выход на ЮБК) 10 минут на автомобиле. Ближайший пляж на Ласпи в 10 минутах , в Форосе в 15 минутах, в Балаклаве в 20 минутах езды на транспорте. До Балаклавского пляжа можно добраться городским автобусом.\n В квартире две спальни, гостиная и кухня. Во дворе есть техническое помещение, где можно хранить велосипеды, коляски и другие нужные вещи. Водоснабжение, канализация, отопление - центральные. Поэтому в квартире всегда тепло и сухо. Дом из инкерманского камня отлично спасает летом от жары. В квартире новые пластиковые окна и надежная дверь. Сделан ремонт - заезжай и живи!\nСогласитесь это нормально?\nРассматривается только наличный расчет.\nВозможна рассрочка платежа!!!\nЗвоните! Все расскажем и покажем!\n\nЛот 225015",16,2023-01-10 11:25:11.144238
517,"1-комн. кв., 65,02 м², 5/10 этаж",Секция 1 • Сдача корпуса 3 кв. 2022,сдача ГК: 4 кв. 2023 года,8 842 720 ₽,136 000 ₽/м²,"[Севастополь, р-н Ленинский, улица Генерала Крейзера, 8]","[6 часов назад, сегодня, 04:59]",https://sevastopol.cian.ru/sale/flat/281147110/,,19,2023-01-10 11:25:11.144238
344,"1-комн. кв., 40,91 м², 1/9 этаж",Сдача корпуса 3 кв. 2025,сдача ГК: 3 кв. 2025 года,5 645 580 ₽,138 000 ₽/м²,"[Севастополь, р-н Гагаринский, улица Братьев Манганари, 30к2]","[2 часа назад, сегодня, 09:25]",https://sevastopol.cian.ru/sale/flat/277315895/,"КВАРТИРЫ НАПРЯМУЮ ОТ ЗАСТРОЙЩИКА! \n\nУЛУЧШЕННЫЙ ПРОЕКТ ЖК ""PORTO FRANCO""\n\nСрок сдачи 3 квартал 2025 \n\n3-комнатная квартира в новом жилом комплексе у МОРЯ - на Западном берегу Камышовой бухты г. Севастополя! \n\nОбщая площадь 73 м², все комнаты раздельные - 17,5 ; 16,5 и 10 м2, просторная кухня 10м2, ванная с окном, застекленная лоджия 4,5 м2. \n\nВ КВАРТИРЕ: \n\n- качественная стяжка пола \n- двухконтурный газовый котел \n- все счетчики \n- радиаторы в комнатах и на кухне \n- стеклопакеты \n- качественная металлическая дверь \n- розетки, выключатели \n\nЖИЛОЙ КОМПЛЕКС НАХОДИТСЯ В 300 МЕТРАХ У МОРЯ: \n\n- Вид прекрасный - 150 градусов видно море! \n- Возможность каждый день гулять у моря и даже купаться - шаговая доступность к морю! \n- Уединенность - нет шумных трасс и магистралей \n- Атмосфера настоящей жизни у моря - морские закаты, уходящие в море корабли, маяк \n\nЖИВИТЕ У МОРЯ ПО-НАСТОЯЩЕМУ! \n\nОТКРЫТЫ ПРЕДПРОДАЖИ! УСПЕВАЙТЕ БРОНИРОВАТЬ! \n\nПолучить всю информацию по остаткам, записаться на просмотр а также, забронировать квартиру можно по телефону и в отделе продаж!",13,2023-01-10 11:25:11.144238


In [7]:
assert len(df)>0

In [8]:
print(len(df))
df = pd.DataFrame(df).dropna().drop_duplicates(['LinkArea','Description',]).reset_index(drop=True)
print(len(df))

532
465


In [9]:
df['place']=place

In [10]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
# df.to_excel(f'{raw_data_path}/cian_{ts}_raw.xlsx',index=False)
df.to_pickle(f'{raw_data_path}/cian_{ts}_raw.pkl')

----

In [11]:
# df = pd.read_excel('data/raw/cian/cian_2022-09-08_13-49_raw.xlsx')

In [12]:
# df

In [13]:
# df['GeoLabel'].apply( lambda s : len(s) ) #', '.join(s) )

# s = ['Севастополь', 'р-н Гагаринский', 'Казачья Бухта мкр', 'Скифия ЖК']
# ', '.join(s)

In [14]:
# from lib.cian import CianDataCleaner

# CianDataCleaner().transform(df)

----

In [15]:
# with open('tmp/cian.html','wt') as f: f.write(html[0])